In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

2023-04-11 22:33:26.166215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 22:33:27.188493: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pardalito/miniconda3/envs/meia/lib/
2023-04-11 22:33:27.188583: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pardalito/miniconda3/envs/meia/lib/
2023-04-11 22:33:27.188592: W tensorflow/com

#### Load the data from a CSV file

In [3]:
df = pd.read_csv('../nlp_data.csv')

In [8]:
for line in df.values[:, :-1]:
    print(line)

[0 7 'October' 2023 15 'Ema' 'high' 10.143 'dewpoint' nan
 'positive contribution' nan 0.0 nan 0.0 2.0 nan]
[1 8 'June' 2023 16 'Ana' 'medium' 2.8 'temperature' nan 'negative impact'
 nan 2.0 1.0 nan nan 0.0]
[2 27 'February' 2023 14 'Ema' 'high' 9.429 'dewpoint' 'temperature'
 'positive contribution' 'positive contribution' nan 0.0 0.0 nan 0.0]
[3 28 'November' 2023 15 'Ana' 'very high' 13.857 'solar radiation' nan
 'significant positive contribution' nan 1.0 0.0 0.0 0.0 nan]
[4 6 'April' 2024 6 'Diana' 'low' 3.429 'solar radiation'
 'hourly precipitation' 'negative impact' 'significant negative impact'
 2.0 nan 0.0 nan nan]
[5 26 'October' 2023 14 'Ana' 'medium' 3.733 'hourly precipitation' nan
 'negative impact' nan nan 2.0 nan 2.0 nan]
[6 14 'April' 2024 11 'Carlos' 'very high' 3.28 'solar radiation'
 'temperature' 'significant positive contribution' 'positive contribution'
 0.0 0.0 nan nan 0.0]
[7 14 'June' 2023 14 'Ema' 'high' 2.48 'dewpoint' 'solar radiation'
 'positive contribu

In [10]:
# Define the maximum length of the input and output sequences
max_input_len = max(len(line.split(' ')) for line in df.values[:, :-1])
max_output_len = max(len(line.split(' ')) for line in df['report'])

AttributeError: 'numpy.ndarray' object has no attribute 'split'

#### Preprocess the data

In [ ]:
input_texts = []
target_texts = []
for i in range(len(df)):
    input_line = df.iloc[i][:-1].to_string(index=False)
    target_line = df.iloc[i]['report']
    input_texts.append(input_line)
    target_texts.append(target_line)

#### Tokenize the input and output sequences

In [ ]:
input_tokenizer = Tokenizer(filters='', lower=False)
input_tokenizer.fit_on_texts(input_texts)
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
input_sequences = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
num_input_tokens = len(input_tokenizer.word_index) + 1

output_tokenizer = Tokenizer(filters='', lower=False)
output_tokenizer.fit_on_texts(target_texts)
target_sequences = output_tokenizer.texts_to_sequences(target_texts)
target_sequences = pad_sequences(target_sequences, maxlen=max_output_len, padding='post')
num_output_tokens = len(output_tokenizer.word_index) + 1

In [ ]:
# Define the model architecture
latent_dim = 256
encoder_inputs = Input(shape=(max_input_len,))
encoder_embedding = Embedding(num_input_tokens, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_output_len,))
decoder_embedding = Embedding(num_output_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_output_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Compile the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

# Train the model
batch_size = 64
epochs = 100
history = model.fit([input_sequences, target_sequences[:, :-1]], target_sequences[:, 1:], batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
# Generate reports
def generate_report(input_sequence):
    encoder_input = input_tokenizer.texts_to_sequences([input_sequence])
    encoder_input = pad_sequences(encoder_input, maxlen=max_input_len, padding='post')
    decoder_input = np.zeros(shape=(1, max_output_len))
    decoder_input[0, 0] = output_tokenizer.word_index['<start>']
    for i in range(1, max_output_len):
        output_tokens = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[0, i] = output_tokens[0, i-1]
        if output_tokens[0, i] == output_tokenizer.word_index['<end>']:
            break
    output_sequence = ' '.join(output_tokenizer.index_word[idx] for idx in decoder_input[0] if idx > 0)
